In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
# inline plotting
%matplotlib inline

In [3]:
from labproject.experiments import Experiment, ScaleDim, ScaleSampleSize
from labproject.utils import get_log_path

In [4]:
from labproject.data import DATASETS, DISTRIBUTIONS
from labproject.metrics import METRICS
from torch.distributions import MultivariateNormal
METRICS.keys()


dict_keys(['mmd_rbf', 'mmd_rbf_median_heuristic', 'mmd_rbf_auto', 'mmd_polynomial', 'mmd_linear_naive', 'mmd_linear', 'c2st_nn', 'c2st_rf', 'c2st_knn', 'gaussian_kl_divergence', 'wasserstein_gauss_squared', 'sliced_wasserstein', 'wasserstein_kuhn', 'wasserstein_sinkhorn'])

In [5]:
from labproject.plotting import generate_palette
color_dict = {"wasserstein": "#cc241d",
              "mmd": "#eebd35",
              "c2st": "#458588",
              "fid": "#8ec07c", 
              "kl": "#8ec07c"}


col_map = {'ScaleSampleSizeKL':'kl', 'ScaleSampleSizeSW':'wasserstein',
           'ScaleSampleSizeMMD':'mmd', 'ScaleSampleSizeC2ST':'c2st',
           'ScaleSampleSizeFID':'fid', 'ScaleDimKL':'kl', 'ScaleDimSW':'wasserstein',
           'ScaleDimMMD':'mmd', 'ScaleDimC2ST':'c2st',
           'ScaleDimFID':'fid',}
from matplotlib.ticker import MaxNLocator



# All multiple datasets

In [6]:
from labproject.data import DATASETS, DISTRIBUTIONS
from labproject.utils import get_cfg, set_seed, get_log_path
from labproject.data import get_dataset
import os
from omegaconf import OmegaConf
from labproject.experiments import *
from labproject.plotting import cm2inch, ensure_zero_ytick
import time
import datetime
import pickle
import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal


print("Running experiments...")
# load the config file
cfg = OmegaConf.load("../../configs/conf_main_scaling_experiment.yaml") 
cfg.running_user = 'sample_size_dimensionality'
seed = cfg.seed

set_seed(seed)
print(f"Seed: {seed}")
print(f"Experiments: {cfg.experiments}") 
print(f"Data: {cfg.data}")


col_dark = {}
col_light = {}
for e, exp_name in enumerate(cfg.experiments):
    col_dark[exp_name] = generate_palette(color_dict[col_map[exp_name]], saturation='dark')[2]
    col_light[exp_name] = generate_palette(color_dict[col_map[exp_name]], saturation='light')[-1]
for e, exp_name in enumerate(cfg.experiments_dim):
    col_dark[exp_name] = generate_palette(color_dict[col_map[exp_name]], saturation='dark')[2]
    col_light[exp_name] = generate_palette(color_dict[col_map[exp_name]], saturation='light')[-1]

color_list = [col_light, col_dark] # make this a list to account for true and shifted

label_true = {}
label_shift = {}
for e, data_name in enumerate(cfg.data):
    label_true[data_name] = "true"
    label_shift[data_name] = "generated"
    
label_list = [label_true, label_shift]
label_list[1]['toy_2d'] = 'approx.'
label_list[1]['random'] = 'shifted'
mapping = {'ScaleSampleSizeKL':'KL', 'ScaleSampleSizeSW':'SW',
        'ScaleSampleSizeMMD':'MMD', 'ScaleSampleSizeC2ST':'C2ST',
        'ScaleSampleSizeFID':'FD', }

mapping = {'ScaleSampleSizeKL':'KL', 'ScaleSampleSizeSW':'SW',
           'ScaleSampleSizeMMD':'MMD', 'ScaleSampleSizeC2ST':'C2ST',
           'ScaleSampleSizeFID':'FD', 'ScaleDimKL':'KL', 'ScaleDimSW':'SW',
           'ScaleDimMMD':'MMD', 'ScaleDimC2ST':'C2ST',
           'ScaleDimFID':'FD',}


# assert cfg.data is list
assert len(cfg.data) == len(cfg.n) == len(cfg.d), "Data, n and d must be lists of the same length"
    
# make comparison plots
fig, axes = plt.subplots(3, 3, figsize=cm2inch((20, 10)), sharex='col')
for ax in axes.flatten():
    # move spines outward
    ax.spines['bottom'].set_position(('outward', 4))
    ax.spines['left'].set_position(('outward', 4))
    ax.locator_params(nbins=3)

for dd, ds in enumerate(cfg.data):

    dataset_fn = get_dataset(cfg.data[dd])
    n_samples = cfg.n[dd]*cfg.runs
    
    dataset_gt = dataset_fn(n_samples,cfg.d[dd])
    dataset_intra = dataset_fn(n_samples,cfg.d[dd])
    
    print(cfg.data[dd], n_samples, cfg.d[dd])
    
    if cfg.data[dd] == 'toy_2d':
        dataset_inter = MultivariateNormal(torch.mean(dataset_gt, axis=0).T,
                                           torch.cov(dataset_gt.T)).sample((n_samples,))
    elif cfg.data[dd] == 'random' and cfg.augmentation[dd]=='mean_shift':
        dataset_inter = dataset_fn(n_samples,cfg.d[dd]) + 1 # just shift the mean by 1
    elif cfg.data[dd] == 'random' and cfg.augmentation[dd]=='one_dim_shift':
        # just shift the first dimension by 0.1
        dataset_inter = dataset_fn(n_samples,cfg.d[dd])
        dataset_inter[:,0] += 1 # just shift the mean of first dim by 1
    elif cfg.data[dd] == 'imagenet_real_embeddings':
        dataset_fn = get_dataset(cfg.augmentation[dd])        
        dataset_inter = dataset_fn(n_samples,cfg.d[dd])

    fig_data, axes_data = plt.subplots(1, 3, figsize=cm2inch((15, 4)), sharex=True, sharey=True)
    axes_data[0].plot(dataset_gt[:1000, 0], dataset_gt[:1000, 1], 'o', label='gt', alpha=0.2, color='C0')
    axes_data[1].plot(dataset_intra[:1000, 0], dataset_intra[:1000, 1], 'o', label='true',  alpha=0.2, color='C2')
    axes_data[2].plot(dataset_inter[:1000, 0], dataset_inter[:1000, 1], 'o', label='approx' ,alpha=0.2, color='C3')
    for ax in axes_data:
        ax.legend()

        
    
    if dd<2: # for the first two datasets, we compare sample sizes
        for e, exp_name in enumerate(cfg.experiments):
            experiment = globals()[exp_name]()
            time_start = time.time()
            dataset1 = dataset_gt
            ax = axes[e,dd]
            for dc, data_comp in enumerate([dataset_intra, dataset_inter]):
                dataset2 = data_comp
                assert dataset1.shape == dataset2.shape, f"Dataset shapes do not match: {dataset1.shape} vs. {dataset2.shape}"
                if mapping[exp_name]  == 'MMD':
                    print(f'MMD {cfg.data[dd]} {dd} {cfg.mmd_bandwidth[dd]}')
                    output = experiment.run_experiment(dataset1=dataset1,
                                                    dataset2=dataset2,
                                                    sample_sizes=cfg.sample_size,
                                                    nb_runs = cfg.runs,
                                                    bandwidth=cfg.mmd_bandwidth[dd])
                else:
                    output = experiment.run_experiment(dataset1=dataset1,
                                                        dataset2=dataset2,
                                                        sample_sizes=cfg.sample_size,
                                                        nb_runs = cfg.runs)
                time_end = time.time()
                print(f"Experiment {exp_name} finished in {time_end - time_start}")
                log_path = get_log_path(cfg, tag=f"_{mapping[exp_name]}_{exp_name}_{cfg.data[dd]}_{dc}", timestamp=False)
                os.makedirs(os.path.dirname(log_path), exist_ok=True)
                experiment.log_results(output, log_path)
                print(f"Numerical results saved to {log_path}")

                experiment.plot_experiment(*output, cfg.data[dd],
                                        ax=ax,
                                        color=color_list[dc][exp_name], 
                                        label=label_list[dc][cfg.data[dd]],
                                        linestyle='-' if dc == 0 else '--',lw=2, marker='o')
                ax.set_ylabel(mapping[exp_name]+f' {cfg.mmd_bandwidth[dd]}' if mapping[exp_name]  == 'MMD' else mapping[exp_name])
                ax.set_xlabel('')
                if mapping[exp_name]  == 'C2ST':
                    ax.set_ylim([0.45,1])
                    ax.set_yticks([0.5,1])
                else:
                    ensure_zero_ytick(ax) # make sure 0 is included in y-ticks
            ax.legend()        
    else: # for the last dataset, we compare dimensions
        for e, exp_name in enumerate(cfg.experiments_dim):
            experiment = globals()[exp_name]()
            time_start = time.time()
            dataset1 = dataset_gt
            ax = axes[e, 2]
            ax.set_xscale('log')
            for dc, data_comp in enumerate([dataset_intra, dataset_inter]):
                dataset2 = data_comp
                assert dataset1.shape == dataset2.shape, f"Dataset shapes do not match: {dataset1.shape} vs. {dataset2.shape}"
                if exp_name == 'ScaleDimMMD':
                    output = experiment.run_experiment(dataset1=dataset1,
                                                    dataset2=dataset2,
                                                    dataset_size = cfg.n[dd], 
                                                    dim_sizes=cfg.dim_sizes,
                                                    nb_runs = cfg.runs_dim, # deterministic
                                                    bandwidth=cfg.mmd_bandwidth[dd])
                    print(f'MMD {cfg.data[dd]} {dd} {cfg.mmd_bandwidth[dd]}')
                else:
                    output = experiment.run_experiment(dataset1=dataset1,
                                                    dataset2=dataset2,
                                                    dataset_size = cfg.n[dd], 
                                                    dim_sizes=cfg.dim_sizes,
                                                    nb_runs = cfg.runs_dim)
                time_end = time.time()
                print(f"Experiment {exp_name} finished in {time_end - time_start}")

                log_path = get_log_path(cfg, tag=f"_{mapping[exp_name]}_{exp_name}_{cfg.data[dd]}_{dc}", timestamp=False)
                os.makedirs(os.path.dirname(log_path), exist_ok=True)
                experiment.log_results(output, log_path)
                print(f"Numerical results saved to {log_path}")
                experiment.plot_experiment(*output,
                    cfg.data[0],
                    ax=ax,
                    color=color_list[dc][exp_name],
                    label=label_list[dc][cfg.data[dd]],
                    linestyle='-' if dc == 0 else '--',lw=2, marker='o'
                )
                ax.set_ylabel(mapping[exp_name]+f' {cfg.mmd_bandwidth[dd]}' if mapping[exp_name]  == 'MMD' else mapping[exp_name])
                ax.set_xlabel('')
                if mapping[exp_name]  == 'C2ST':
                    ax.set_ylim([0.45,1])
                    ax.set_yticks([0.5,1])
                else:
                    ensure_zero_ytick(ax) # make sure 0 is included in y-ticks
                
            ax.legend()

axes[-1, -1].set_xlabel('dimensions')
axes[-1,0].set_xlabel('sample size')
axes[-1,1].set_xlabel('sample size')

fig.tight_layout()
print(f"Plots saved to {cfg.data}.png")
fig.savefig(f"./results/plots/{cfg.exp_log_name}_metric_comparison_sample_size_and_dimensionality_{cfg.mmd_bandwidth}_5_folds_all.png", dpi=300)
fig.savefig(f"./results/plots/{cfg.exp_log_name}_metric_comparison_sample_size_and_dimensionality_{cfg.mmd_bandwidth}_5_folds_all.pdf", dpi=300)

print("Finished running experiments.")

Running experiments...
Seed: 0
Experiments: ['ScaleSampleSizeSW', 'ScaleSampleSizeC2ST', 'ScaleSampleSizeMMD']
Data: ['toy_2d', 'random', 'random']
toy_2d 50000 2


/tmp/ipykernel_428418/899309173.py:80: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3637.)
  dataset_inter = MultivariateNormal(torch.mean(dataset_gt, axis=0).T,


Experiment ScaleSampleSizeSW finished in 0.1651298999786377
Numerical results saved to results/sample_size_dimensionality/sample_size_dimensionality_SW_ScaleSampleSizeSW_toy_2d_0_.pkl
Experiment ScaleSampleSizeSW finished in 0.3243546485900879
Numerical results saved to results/sample_size_dimensionality/sample_size_dimensionality_SW_ScaleSampleSizeSW_toy_2d_1_.pkl
Experiment ScaleSampleSizeC2ST finished in 58.46237063407898
Numerical results saved to results/sample_size_dimensionality/sample_size_dimensionality_C2ST_ScaleSampleSizeC2ST_toy_2d_0_.pkl
Experiment ScaleSampleSizeC2ST finished in 147.89045429229736
Numerical results saved to results/sample_size_dimensionality/sample_size_dimensionality_C2ST_ScaleSampleSizeC2ST_toy_2d_1_.pkl
MMD toy_2d 0 1
Experiment ScaleSampleSizeMMD finished in 1.8278639316558838
Numerical results saved to results/sample_size_dimensionality/sample_size_dimensionality_MMD_ScaleSampleSizeMMD_toy_2d_0_.pkl
MMD toy_2d 0 1
Experiment ScaleSampleSizeMMD finish